# Image Classification using PyTorch Lightning

Reference:
    https://pytorch-lightning.readthedocs.io/en/latest/

In [32]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
#from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms, datasets
import torchvision.models as models

import pytorch_lightning as pl
from pytorch_lightning import Trainer

torch.__version__, torchvision.__version__, pl.__version__

('1.7.1', '0.8.2', '1.1.3')

# CUDA が利用できる環境なのかチェック

In [33]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# 1. MNIST

Dataset: MNIST

In [5]:
transform = transforms.Compose([
    transforms.ToTensor()
])

train_val = torchvision.datasets.MNIST(root='data', train=True, download=True, transform=transform)
test = torchvision.datasets.MNIST(root='data', train=False, download=True, transform=transform)

n_train = int(len(train_val) * 0.8)
n_val = len(train_val) - n_train

torch.manual_seed(0)

train, val = torch.utils.data.random_split(train_val, [n_train, n_val])

batch_size = 32

train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

|          | 0/? [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw


|          | 0/? [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw


|          | 0/? [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw


|          | 0/? [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw
Processing...


C:\Users\dahat\anaconda3\envs\pytorch-lightning\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [6]:
class LearningModel(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        
        self.conv = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=(3,3))
        self.fc = nn.Linear(507, 10)
        
        self.train_acc = pl.metrics.Accuracy()
        self.val_acc = pl.metrics.Accuracy()
        self.test_acc = pl.metrics.Accuracy()
        
    def forward(self, x):
        h = self.conv(x)
        h = F.max_pool2d(h, kernel_size=(2,2), stride=2)
        h = F.relu(h)
        h = h.view(-1, 507)
        h = self.fc(h)
        return h

    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        self.log('tran_acc', self.train_acc(y, t), on_step=True, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_acc', self.val_acc(y, t), on_step=False, on_epoch=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        self.log('test_acc', self.test_acc(y, t), on_step=False, on_epoch=True)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.001, momentum=0.9)
        return optimizer

In [8]:
pl.seed_everything(0)
model = LearningModel()
trainer = pl.Trainer(max_epochs=10, gpus=1)
trainer.fit(model, train_loader, val_loader)

result = trainer.test(test_dataloaders=test_loader)
result

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type     | Params
---------------------------------------
0 | conv      | Conv2d   | 30    
1 | fc        | Linear   | 5.1 K 
2 | train_acc | Accuracy | 0     
3 | val_acc   | Accuracy | 0     
4 | test_acc  | Accuracy | 0     
---------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params


Validation sanity check: |                                                                       | 0/? [00:00<…

Training: |                                                                                      | 0/? [00:00<…

Validating: |                                                                                    | 0/? [00:00<…

Validating: |                                                                                    | 0/? [00:00<…

...

Validating: |                                                                                    | 0/? [00:00<…

Validating: |                                                                                    | 0/? [00:00<…

Testing: |                                                                                       | 0/? [00:00<…

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.9301, device='cuda:0'),
 'test_loss': tensor(0.2435, device='cuda:0')}
--------------------------------------------------------------------------------


[{'test_loss': 0.2434733361005783, 'test_acc': 0.9301000237464905}]

# 2. Fine Turning

Dataset: CIFAR10

Model: ResNet18

Please see provisioned model
https://pytorch.org/docs/stable/torchvision/models.html

In [43]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

train_val = datasets.CIFAR10('data', train=True, download=True, transform=transform)
test = datasets.CIFAR10('data', train=False, download=True, transform=transform)

torch.manual_seed(0)

n_train = int(len(train_val) * 0.8)
n_val = len(train_val) - n_train
train, val = torch.utils.data.random_split(train_val, [n_train, n_val])

batch_size = 256

train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True, num_workers=8)
val_loader = torch.utils.data.DataLoader(val, batch_size, num_workers=8)
test_loader = torch.utils.data.DataLoader(test, batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [62]:
class FineTurningModel(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        
        self.feature_extractor = models.resnet18(pretrained=True)
        for param in self.feature_extractor.parameters():
            param.requires_grad = False
        
        self.fc = nn.Linear(1000, 10)
        
        self.train_acc = pl.metrics.Accuracy()
        self.val_acc = pl.metrics.Accuracy()
        self.test_acc = pl.metrics.Accuracy()
        
    def forward(self, x):
        h = self.feature_extractor(x)
        h = self.fc(h)
        return h

    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        self.log('tran_acc', self.train_acc(y, t), on_step=True, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_acc', self.val_acc(y, t), on_step=False, on_epoch=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        self.log('test_acc', self.test_acc(y, t), on_step=False, on_epoch=True)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.001, momentum=0.9)
        return optimizer

In [64]:
pl.seed_everything(0)
model = FineTurningModel()
model

FineTurningModel(
  (feature_extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, aff

In [65]:
trainer = pl.Trainer(max_epochs=10, gpus=1)
trainer.fit(model, train_loader, val_loader)

result = trainer.test(test_dataloaders=test_loader)
result

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type     | Params
-----------------------------------------------
0 | feature_extractor | ResNet   | 11.7 M
1 | fc                | Linear   | 10.0 K
2 | train_acc         | Accuracy | 0     
3 | val_acc           | Accuracy | 0     
4 | test_acc          | Accuracy | 0     
-----------------------------------------------
10.0 K    Trainable params
11.7 M    Non-trainable params
11.7 M    Total params


Validation sanity check: |                                                                       | 0/? [00:00<…

Training: |                                                                                      | 0/? [00:00<…

Validating: |                                                                                    | 0/? [00:00<…

Validating: |                                                                                    | 0/? [00:00<…

...

Validating: |                                                                                    | 0/? [00:00<…

Validating: |                                                                                    | 0/? [00:00<…

Testing: |                                                                                       | 0/? [00:00<…

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.7693, device='cuda:0'),
 'test_loss': tensor(0.6685, device='cuda:0')}
--------------------------------------------------------------------------------


[{'test_loss': 0.6685366034507751, 'test_acc': 0.7692999839782715}]

# 3. Own Image with Fine Turning

Dataset - Dog and Cat:
https://www.kaggle.com/c/dogs-vs-cats/overview

In [91]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

dataset = torchvision.datasets.ImageFolder('data/cat_dog', transform)
dataset

Dataset ImageFolder
    Number of datapoints: 25200
    Root location: data/cat_dog
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [92]:
torch.manual_seed(0)

n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

batch_size = 256

train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True, num_workers=8)
val_loader = torch.utils.data.DataLoader(val, batch_size, num_workers=8)
test_loader = torch.utils.data.DataLoader(test, batch_size)

In [93]:
class OwnFineTurningModel(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        
        self.feature_extractor = models.resnet18(pretrained=True)
        for param in self.feature_extractor.parameters():
            param.requires_grad = False
        
        self.fc = nn.Linear(1000, 2)
        
        self.train_acc = pl.metrics.Accuracy()
        self.val_acc = pl.metrics.Accuracy()
        self.test_acc = pl.metrics.Accuracy()
        
    def forward(self, x):
        h = self.feature_extractor(x)
        h = self.fc(h)
        return h

    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        self.log('tran_acc', self.train_acc(y, t), on_step=True, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_acc', self.val_acc(y, t), on_step=False, on_epoch=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        self.log('test_acc', self.test_acc(y, t), on_step=False, on_epoch=True)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.001, momentum=0.9)
        return optimizer

In [94]:
pl.seed_everything(0)
model = OwnFineTurningModel()
model

OwnFineTurningModel(
  (feature_extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, 

In [95]:
trainer = pl.Trainer(max_epochs=10, gpus=1)
trainer.fit(model, train_loader, val_loader)

result = trainer.test(test_dataloaders=test_loader)
result

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type     | Params
-----------------------------------------------
0 | feature_extractor | ResNet   | 11.7 M
1 | fc                | Linear   | 2.0 K 
2 | train_acc         | Accuracy | 0     
3 | val_acc           | Accuracy | 0     
4 | test_acc          | Accuracy | 0     
-----------------------------------------------
2.0 K     Trainable params
11.7 M    Non-trainable params
11.7 M    Total params


Validation sanity check: |                                                                       | 0/? [00:00<…

Training: |                                                                                      | 0/? [00:00<…

Validating: |                                                                                    | 0/? [00:00<…

Validating: |                                                                                    | 0/? [00:00<…

...

Validating: |                                                                                    | 0/? [00:00<…

Validating: |                                                                                    | 0/? [00:00<…

Testing: |                                                                                       | 0/? [00:00<…

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.9796, device='cuda:0'),
 'test_loss': tensor(0.0553, device='cuda:0')}
--------------------------------------------------------------------------------


[{'test_loss': 0.05527614429593086, 'test_acc': 0.9795634746551514}]

## Todo

- GPU有無
- テスト画像の推論処理
- FineTurning のクラス数を自動抽出
- Hyperparameter

    https://pytorch-lightning.readthedocs.io/en/latest/hyperparameters.html

